In [ ]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings
import psycopg2

# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.options.display.float_format = '{:.3f}'.format

df_sc = statcast(start_dt='2024-01-01', end_dt='2024-09-30')



This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 200/200 [02:07<00:00,  1.56it/s]
/opt/homebrew/anaconda3/lib/python3.11/site-packages/pybaseball/statcast.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat(dataframe_list, axis=0).convert_dtypes(convert_string=False)


In [ ]:

# Step 1: Your existing unique count
df_unique = df_sc[['game_year', 'pitcher', 'p_throws', 'game_pk', 'at_bat_number']].drop_duplicates()
result = df_unique.groupby(['game_year', 'pitcher', 'p_throws']).size().reset_index(name='num_at_bats')

# Step 2: Pivot to check counts for both stands
pivot = result.pivot(index=['game_year', 'pitcher'], columns='p_throws', values='num_at_bats').fillna(0)

# Step 3: Apply logic for labeling
def label_row(row):
    if row.get('L', 0) > 5 and row.get('R', 0) > 5:
        return 'B'
    elif row.get('L', 0) > row.get('R', 0):
        return 'L'
    else:
        return 'R'

pivot['throws'] = pivot.apply(label_row, axis=1)

# Step 4: Merge back with your original result
final_result = result.merge(pivot[['throws']], on='pitcher')

pitcher_throws_info = final_result[['game_year', 'pitcher', 'throws']].drop_duplicates()

pitcher_throws_info.to_csv('pitcher_info.csv', index=False)
pitcher_throws_info

# psql -h piedmont-db.cp6uayweg6ql.us-east-2.rds.amazonaws.com -U postgres -d postgres -p 5432
# \COPY pitcher_info(game_year, pitcher, throws) FROM '/Users/loganmottley/Desktop/Projects/go-baseball/data/pitcher_info.csv' DELIMITER ',' CSV HEADER;

,game_year,pitcher,throws
0,2024,434378,R
1,2024,445276,R
2,2024,445926,R
3,2024,448179,L
4,2024,450203,R
...,...,...,...
1173,2024,814005,R
1174,2024,814088,R
1175,2024,814217,R
1176,2024,814280,R


In [ ]:


pitcher_count_pitch_freq = (
    df_sc.groupby(['game_year', 'player_name', 'pitcher', 'stand', 'pitch_type', 'balls', 'strikes'])
         .size()
         .reset_index(name='count')
)

# Then calculate total_count for each pitcher + stand group
pitcher_count_pitch_freq['total_count'] = pitcher_count_pitch_freq.groupby(['pitcher', 'stand', 'balls', 'strikes'])['count'].transform('sum')

# Calculate frequency
pitcher_count_pitch_freq['frequency'] = pitcher_count_pitch_freq['count'] / pitcher_count_pitch_freq['total_count']

# Sort by count
pitcher_count_pitch_freq = pitcher_count_pitch_freq.sort_values('count', ascending=False)

# Optional: display top 10
pitcher_count_pitch_freq



pitcher_count_pitch_freq.to_csv('pitcher_count_pitch_freq.csv', index=False)


# psql -h piedmont-db.cp6uayweg6ql.us-east-2.rds.amazonaws.com -U postgres -d postgres -p 5432
# \COPY pitcher_count_pitch_freq(game_year, player_name, pitcher, stand, pitch_type, balls, strikes, count, total_count, frequency) FROM '/Users/loganmottley/Desktop/Projects/go-baseball/data/pitcher_count_pitch_freq.csv' DELIMITER ',' CSV HEADER;

